In [9]:
# !pip install qiskit-nature

In [47]:
from math import pi
import json
from pathlib import Path

import numpy as np
from dotenv import load_dotenv
import rustworkx as rx
from qiskit_nature.second_q.hamiltonians.lattices import (
    BoundaryCondition,
    HyperCubicLattice,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
    SquareLattice,
    TriangularLattice,
)
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel
from qiskit_nature.second_q.problems import LatticeModelProblem
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import QAOAAnsatz
from bio_bundles import registrar
from math import sqrt
from typing import *
from qiskit.circuit.library import QAOAAnsatz
from process_bigraph import Process, Step
from process_bigraph import pp
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit.primitives import Sampler
from qiskit.quantum_info import Pauli
from qiskit.result import QuasiDistribution
from qiskit_algorithms import QAOA as QAOASolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals

from bio_bundles import registrar


CORE = registrar.core


load_dotenv("../.env")

True

In [48]:
from process_bigraph import Process, Step


class VariableReduction(Process):
    """Run reduce_variables() workflow here"""
    pass


class VariableChecker(Step):
    """takes N as input state and checks if N == Ncrit"""
    pass


from qiskit.quantum_info import SparsePauliOp
def build_max_cut_paulis(graph: rx.PyGraph) -> list[tuple[str, float]]:
    """Convert the graph to Pauli list.

    This function does the inverse of `build_max_cut_graph`
    """
    pauli_list = []
    for edge in list(graph.edge_list()):
        paulis = ["I"] * len(graph)
        paulis[edge[0]], paulis[edge[1]] = "Z", "Z"

        weight = graph.get_edge_data(edge[0], edge[1])

        pauli_list.append(("".join(paulis)[::-1], weight))

    return pauli_list


class Grover(Process):
    config_schema = {}

    def __init__(self, config, core):
        super().__init__(config, core)


class RQAOA(Process):
    config_schema = {
        'n_nodes': 'integer',
        'edge_list': 'list',
    }

    def __init__(self, config, core):
        super().__init__(config, core)
        self.n_nodes = self.config.get('n_nodes')
        self.graph = rx.PyGraph()

        # add nodes
        self.graph.add_nodes_from(
            np.arange(0, self.n_nodes, 1)
        )

        # add edges from parameters
        self.graph.add_edges_from(
            self.config.get('edge_list')
        )
        self.max_cut_paulis = build_max_cut_paulis(self.graph)
        self.H_cost = SparsePauliOp.from_list(self.max_cut_paulis)
        self.circuit = QAOAAnsatz(cost_operator=self.H_cost, reps=2)
        self.measure()

    def measure(self):
        return self.circuit.measure_all()



class MolecularQAOA(Process):
    config_schema = {
        'nuclei': 'list[list[float]]'
    }

    def __init__(self, config, core):
        super().__init__(config, core)

        # position vectors of all nuclei involved in solving
        self.R = self.config['nuclei']

In [49]:
# from bio_bundles import registrar
# CORE = registrar.core
# 
# rqaoa = RQAOA(
#     config={
#         'n_nodes': 5,
#         'edge_list': [(0, 1, 1.0), (0, 2, 1.0), (0, 4, 1.0), (1, 2, 1.0), (2, 3, 1.0), (3, 4, 1.0)] 
#     },
#     core=CORE
# )

In [50]:
# rqaoa.circuit.draw('mpl')

In [13]:
"""R is an array of arrays of shape: (n, 3)
    where n is the number of nuclei involved in the hamiltonian, and 
    3 represent position vector dimesions (x, y, z) for each n
"""

'R is an array of arrays of shape: (n, 3)\n    where n is the number of nuclei involved in the hamiltonian, and \n    3 represent position vector dimesions (x, y, z) for each n\n'

True

In [46]:
# TODO: create process using local-only QAOA!
def rustworkx_graph_setup(w: list[list[float]]):
    num_nodes = len(w)
    w = np.array(w)
    
    # Create graph from adjacency matrix
    edges = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if w[i, j] > 0:  # Add edges where weight > 0
                edges.append((i, j, w[i, j]))
    
    G = rx.PyGraph()
    G.add_nodes_from(range(num_nodes))  # Add nodes
    G.add_edges_from(edges)  # Add edges with weights
    return G


def configure_runtime(circuit: QAOAAnsatz):
    import os
    from qiskit_ibm_runtime import QiskitRuntimeService
    from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
    
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=os.getenv("IBM_QUANTUM_TOKEN"), overwrite=True, set_as_default=True)
    service = QiskitRuntimeService(channel='ibm_quantum')
    backend = service.least_busy(min_num_qubits=127)
    print(backend)
    
    # Create pass manager for transpilation
    pm = generate_preset_pass_manager(optimization_level=3,
                                        backend=backend)
    
    candidate_circuit = pm.run(circuit)
    candidate_circuit.draw('mpl', fold=False, idle_wires=False)
    
    return backend, pm, candidate_circuit


def get_node_pairs(nodes: list) -> list[tuple]:
    from itertools import combinations
    return list(combinations(nodes, 2))


def random_weight() -> float:
    n = np.random.rand()
    return n**n/n if n > 0.5 else n**n*n


def get_weights(node_pairs: list[tuple], generator: Callable) -> list[tuple[int, int, float]]:
    # node_list = [(0, 1), (0, 2), (0, 4), (1, 2), (2, 3), (3, 4)]
    return list(map(
        lambda nodes: (*nodes, generator()),
        node_pairs
    ))


def get_random_weights(node_pairs: list[tuple]):
    return get_weights(node_pairs, random_weight)


def random_connection():
    n = np.random.rand()
    return 1.0 if n > 0.5 else 0.0


def random_adjacency_matrix(num_nodes: int):
    r = range(num_nodes)
    w = [[random_connection() for _ in r] for _ in r]
    return np.array(w)
    

def draw_graph(graph, node_size=600):
    from rustworkx.visualization import mpl_draw as _draw_graph
    return _draw_graph(graph, node_size=node_size, with_labels=True)


def initialize_graph_k(n_nodes_k: int) -> rx.PyGraph:
        """
        Initialize new graph for iteration k.

        :params n_nodes_k: (`int`) Number of nodes (variables) at iteration k for the N-var qaoa solution.

        :rtype: `rustworkx.PyGraph` instance parameterized by n nodes.
        """
        graph_k = rx.PyGraph()

        # add num_nodes for iteration k (n_variables for k)
        nodes_k = np.arange(0, n_nodes_k, 1).tolist()
        graph_k.add_nodes_from(nodes_k)

        # add edge list for iteration k
        node_pairs = get_node_pairs(nodes_k)
        edges_k = get_weights(node_pairs, random_weight)
        graph_k.add_edges_from(edges_k)
        return graph_k
    
    
class GroverProcess(Process):
    config_schema = {
        "target": "string"
    }

    def __init__(self, config, core):
        super().__init__(config, core)


class QAOA(Process):
    config_schema = {
        "n_variables": "integer",
        "random_seed": "integer"
        # "edge_list": "list[tuple[float]]",  # in the format: [(node_a, node_b, weight/is_connected(1 or 0)), ...]
    }

    def __init__(self, config, core):
        super().__init__(config, core)
        self.n_variables = self.config["n_variables"]
        self.random_seed = self.config.get("random_seed", 10598)

        # initial params for probablistic terms
        self.initial_gamma = np.pi
        self.initial_beta = np.pi/2
        self.init_params = [self.initial_gamma, self.initial_beta, self.initial_gamma, self.initial_beta]

    def initial_state(self):
        return {
            "bitstring": [0 for _ in range(self.n_variables)],
            "n_nodes": self.n_variables
        }

    def inputs(self):
        return {
            "n_nodes": "integer",
            "adjacentcy_matrix": "list[list[float]]"
        }

    def outputs(self):
        return {
            "bitstring": "list[integer]",
            "n_nodes": "integer"
        }

    def update(self, inputs, interval):
        # graph = initialize_graph_k(n_nodes_k)

        # initialize graph
        n_nodes_k = inputs.get("n_nodes")
        w = np.array(
            inputs.get("adjacentcy_matrix")
        )
        G = nx.from_numpy_array(w)

        # get quantum operator
        qubit_op, offset = get_operator(w, n_nodes_k)

        # set up optimizer and sampler
        optimizer = COBYLA()
        sampler = Sampler()
        algorithm_globals.random_seed = self.random_seed

        # perform qaoa
        qaoa = QAOASolver(sampler, optimizer, reps=2)

        # extract bitstring
        result = qaoa.compute_minimum_eigenvalue(qubit_op)
        bitstring_k = sample_most_likely(result.eigenstate)

        return {
            "bitstring": bitstring_k.tolist(),
            "n_nodes": n_nodes_k - 1
        }


def bitfield(n, L):
    result = np.binary_repr(n, L)
    return [int(digit) for digit in result]


def get_operator(weight_matrix, num_nodes: int):
    r"""Generate Hamiltonian for the graph partitioning
    Notes:
        Goals:
            1 Separate the vertices into two set of the same size.
            2 Make sure the number of edges between the two set is minimized.
        Hamiltonian:
            H = H_A + H_B
            H_A = sum\_{(i,j)\in E}{(1-ZiZj)/2}
            H_B = (sum_{i}{Zi})^2 = sum_{i}{Zi^2}+sum_{i!=j}{ZiZj}
            H_A is for achieving goal 2 and H_B is for achieving goal 1.
    Args:
        weight_matrix: Adjacency matrix.
        num_nodes: N vars at iteration k.
    Returns:
        Operator for the Hamiltonian
        A constant shift for the obj function.
    """
    num_nodes = len(weight_matrix)
    pauli_list = []
    coeffs = []
    shift = 0

    for i in range(num_nodes):
        for j in range(i):
            if weight_matrix[i, j] != 0:
                x_p = np.zeros(num_nodes, dtype=bool)
                z_p = np.zeros(num_nodes, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, x_p)))
                coeffs.append(-0.5)
                shift += 0.5

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                x_p = np.zeros(num_nodes, dtype=bool)
                z_p = np.zeros(num_nodes, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, x_p)))
                coeffs.append(1.0)
            else:
                shift += 1

    return SparsePauliOp(pauli_list, coeffs=coeffs), shift


def sample_most_likely(state_vector):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.

    Returns:
        Binary string as an array of ints.
    """
    if isinstance(state_vector, QuasiDistribution):
        values = list(state_vector.values())
    else:
        values = state_vector
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = bitfield(k, n)
    x.reverse()
    return np.asarray(x)


In [53]:
n_variables = 5
w_initial = [[0.0, 1.0, 1.0, 1.0, 1.0], [1.0, 0.0, 1.0, 1.0, 1.0], [1.0, 1.0, 0.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.0, 1.0], [1.0, 1.0, 1.0, 1.0, 0.0]]
input_state = {
    "n_nodes": n_variables,
    "adjacentcy_matrix": w_initial
}


qaoa = QAOA(
    config={
        "n_variables": n_variables,
    },
    core=CORE
)

outputs = {}

for i, n in enumerate(list(range(4))):  
    output_state_i = qaoa.update(input_state, 1)
    outputs[str(i)] = output_state_i

/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/ipykernel_21011/524789107.py:157: DeprecationWarning:

The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.

/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/ipykernel_21011/524789107.py:157: DeprecationWarning:

The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.

/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/ipykernel_21011/524789107.py:157: DeprecationWarning:

The class ``qiskit.primitives.sample

In [54]:
outputs

{'0': {'bitstring': [1, 1, 0, 0, 0], 'n_nodes': 4},
 '1': {'bitstring': [1, 1, 0, 0, 0], 'n_nodes': 4},
 '2': {'bitstring': [1, 1, 0, 0, 0], 'n_nodes': 4},
 '3': {'bitstring': [1, 1, 0, 0, 0], 'n_nodes': 4}}